In [1]:
import pandas as pd
raw_data_df = pd.read_csv('vnexpress_foods_detail.csv')

# Mask
mask = raw_data_df.apply(
    lambda row: row['step'] in ([], "[]"),
    axis=1
)
# Create a new DataFrame with the filtered rows
empty_rows_df = raw_data_df[mask].reset_index(drop=True)

display(empty_rows_df)

print(f"{len(empty_rows_df)}")


,link,type_of_food,title,description,author_name,cook_time,num_of_people,calories,num_of_ingredients,ingredients,step,note,post_date
0,https://vnexpress.net/cach-che-bien-vai-mon-au...,Món ngon hàng ngày,Cách chế biến vài món Âu đơn giản,Các món ăn Âu dưới đây mang phong cách pha trộ...,NaN,NaN,NaN,NaN,9,"['Tôm', 'Nghêu', 'Mực', 'Mỳ Ý', 'Thịt bò', 'Cơ...",[],[],"Thứ hai, 22/11/2021, 10:22 (GMT+7)"
1,https://vnexpress.net/nhung-mon-ngon-cho-mua-h...,Món ngon hàng ngày,Những món ngon cho mùa heo may,"Canh rong biển sườn non, bò kho ớt, bí đỏ xào ...",NaN,NaN,NaN,NaN,6,"['Sườn non', 'Rong biển', 'Bò', 'Bí đỏ', 'Nghê...",[],[],"Thứ năm, 18/11/2021, 15:44 (GMT+7)"
2,https://vnexpress.net/3-mon-ngon-de-lam-431145...,Món ngon hàng ngày,3 món ngon dễ làm,Những món ăn hấp dẫn và đầy đủ dưỡng chất với ...,NaN,NaN,NaN,NaN,6,"['Khoai tây', 'Thịt bò', 'Cá hú', 'Tiêu xanh',...",[],[],"Thứ năm, 18/11/2021, 15:38 (GMT+7)"
3,https://vnexpress.net/bien-tau-voi-mon-canh-43...,Món ngon hàng ngày,Biến tấu với món canh,Canh bắp và canh mướp trứng đều vừa giàu dinh ...,NaN,NaN,NaN,NaN,3,"['Ngô ngọt', 'Mướp hương', 'Trứng gà']",[],[],"Thứ tư, 17/11/2021, 16:16 (GMT+7)"
4,https://vnexpress.net/cac-mon-an-dam-huong-que...,Món ngon hàng ngày,Các món ăn đậm hương quê,"Mang hương vị đặc trưng của biển, những món ăn...",NaN,NaN,NaN,NaN,7,"['Ốc bươu ta', 'Giò sống', 'Gạo nếp', 'Cá rô đ...",[],[],"Thứ tư, 17/11/2021, 15:57 (GMT+7)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,https://vnexpress.net/sinh-to-tu-rau-qua-43112...,"Món tráng miệng, giải khát",Sinh tố từ rau quả,"Ngon, có lợi cho sức khỏe là công dụng c...",NaN,NaN,NaN,NaN,7,"['Bắp cải', 'Táo', 'Rau cần', 'Cà rốt', 'Cần t...",[],[],"Thứ ba, 26/10/2021, 11:18 (GMT+7)"
65,https://vnexpress.net/mon-trang-mieng-4311313....,"Món tráng miệng, giải khát",Món tráng miệng,Dưa hấu trộn mật ong và đào ngâm rượu vang là ...,NaN,NaN,NaN,NaN,4,"['Dưa hấu', 'Mật ong', 'Đào', 'Rượu vang trắng']",[],[],"Thứ ba, 26/10/2021, 11:01 (GMT+7)"
66,https://vnexpress.net/rau-cu-kho-chao-4325208....,Thực đơn cho ngày nắng nóng,Rau củ kho chao,Món rau củ có vị béo ngậy và thơm đặc trưng củ...,NaN,NaN,NaN,NaN,5,"['Chao ớt ngon: 3-4 viên kèm nước ngâm chao', ...",[],[],"Thứ tư, 7/7/2021, 00:11 (GMT+7)"
67,https://vnexpress.net/mot-so-mon-an-sang-ngon-...,Bữa sáng đơn giản,Một số món ăn sáng ngon,Nhịp sống vội vã đôi khi làm bạn quên bữa ăn q...,NaN,NaN,NaN,NaN,9,"['Thịt xông khói', 'Dưa vàng', 'Thịt băm', 'Mì...",[],[],"Thứ hai, 22/11/2021, 10:39 (GMT+7)"


69


### crawl data from multi-item websites

In [2]:
import requests
from bs4 import BeautifulSoup
from typing import List
import re

In [3]:
def safe_text(node):
    return node.get_text(strip=True) if node else None

In [4]:
def normalize_ingredient_block(text: str) -> List[str]:
    if not text:
        return []
    text = text.replace("Gia vị:", ",")
    items = [t.strip(" .;") for t in text.split(",")]
    return [i for i in items if i]

In [5]:
def normalize_steps_block(raw_steps: List[str]) -> List[str]:
    """
    Normalize the cooking step list:
    - Merge all text from raw_steps
    - Split by '-' or new line
    - If the paragraph is long without bullets, split by '. '
    - Remove empty spaces, strip extra punctuation
    - Number 'Step 1:', 'Step 2:', ...
    """
    flat = []
    for s in raw_steps:
        if not s:
            continue
        s = s.strip()
        # Prioritize separation by bullet '-', or new line
        parts = re.split(r"[-•\n]+", s)
        for p in parts:
            p = p.strip(" .;:-")
            if p and not p.lower().startswith("cách làm"):
                flat.append(p)

    # If there is only 1 long paragraph → split by sentence
    if len(flat) == 1 and ". " in flat[0]:
        sentences = [sen.strip(" .;") for sen in flat[0].split(". ") if sen.strip()]
        flat = sentences

    return [f"Bước {i+1}: {p}" for i, p in enumerate(flat)]

In [6]:
def get_food_detail(url: str, category: str) -> List[dict]:
    try:
        resp = requests.get(url, timeout=10)
        resp.encoding = "utf-8"
        soup = BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"[ERROR] {url}: {e}")
        return []

    # ==================== Metadata to all items on the page ====================
    description_tag = soup.find("p", class_="description")
    author_tag = soup.select_one("div.name b")
    date_tag = soup.find("span", class_="date")

    items_tag = soup.find("div", class_="status flex")
    time, num_of_people, calories = None, None, None
    
    if items_tag:
        for item in items_tag.find_all("p", class_="itemt"):
            text = item.get_text(strip=True)
            if "phút" in text or "giờ" in text:
                time = text
            elif "người" in text:
                num_of_people = text
            elif "kcal" in text or "calo" in text:
                calories = text

    notes = []
    extra_info = soup.find("div", class_="extra_info")
    if extra_info:
        note_section = extra_info.find("ol") or extra_info.find("ul")
        if note_section:
            for li in note_section.find_all("li"):
                t = li.get_text(separator=" ", strip=True)
                if t:
                    notes.append(t)

    # ==================== MULTI-RECIPE PARSING ====================
    content = soup.find("div", class_="fck_detail")
    if not content:
        return []

    recipes = []
    current_title = None
    current_ingredients_blocks = []  # each block is a raw str, later normalized
    current_steps_blocks = []  # raw blocks

    # iterate over p elements in content to preserve order
    for elem in content.find_all("p"):
        classes = elem.get("class", [])
        text = elem.get_text(separator=" ", strip=True)
        if not text:
            continue

        if "SubTitle" in classes:
            # finalize previous
            if current_title:
                ingredients_list = []
                for ib in current_ingredients_blocks:
                    ingredients_list.extend(normalize_ingredient_block(ib))
                steps_list = normalize_steps_block(current_steps_blocks)

                result = {
                    "link": url,
                    "type_of_food": category,
                    "title": current_title,
                    "description": safe_text(description_tag),
                    "author_name": safe_text(author_tag),
                    "cook_time": time,
                    "num_of_people": num_of_people,
                    "calories": calories,
                    "num_of_ingredients": len(ingredients_list) if ingredients_list else None,
                    "ingredients": ingredients_list,
                    "step": steps_list,
                    "note": notes,
                    "post_date": safe_text(date_tag),
                }
                recipes.append(result)

            # start new recipe
            current_title = text
            current_ingredients_blocks = []
            current_steps_blocks = []

        elif "Normal" in classes:
            low = text.lower()
            # identify ingredient block
            if low.startswith("nguyên liệu"):
                # keep the whole line to be normalized later
                _, sep, rest = text.partition(":")
                if sep:
                    current_ingredients_blocks.append(rest.strip())
                else:
                    # no colon — keep entire line
                    current_ingredients_blocks.append(text)
            elif low.startswith("cách làm") or low.startswith("cách chế biến") or low.startswith("thực hiện"):
                # extract after colon if present
                _, sep, rest = text.partition(":")
                if sep and rest.strip():
                    current_steps_blocks.append(rest.strip())
                else:
                    # maybe steps start in subsequent Normal paragraphs
                    # append empty marker to signal steps started
                    current_steps_blocks.append("")
            else:
                # context sensitive: if we already saw steps section, append there,
                # otherwise treat as ingredient continuation
                if current_steps_blocks:
                    current_steps_blocks.append(text)
                else:
                    current_ingredients_blocks.append(text)

    # finalize last recipe
    if current_title:
        ingredients_list = []
        for ib in current_ingredients_blocks:
            ingredients_list.extend(normalize_ingredient_block(ib))
        steps_list = normalize_steps_block(current_steps_blocks)

        result = {
            "link": url,
            "type_of_food": category,
            "title": current_title,
            "description": safe_text(description_tag),
            "author_name": safe_text(author_tag),
            "cook_time": time,
            "num_of_people": num_of_people,
            "calories": calories,
            "num_of_ingredients": len(ingredients_list) if ingredients_list else None,
            "ingredients": ingredients_list,
            "step": steps_list,
            "note": notes,
            "post_date": safe_text(date_tag),
        }
        recipes.append(result)

    return recipes


In [27]:
all_foods_details = []

for index, row in empty_rows_df.iterrows():
    url = row['link']
    category = row['type_of_food']
    food_details = get_food_detail(url, category)
    if food_details:
        print(f"[INFO] Fetched {len(food_details)} recipes from {url}")
        for detail in food_details:
            print(detail['title'])
            all_foods_details.append(detail)
    else:
        print(f"[WARN] No details found for {url}")


[INFO] Fetched 4 recipes from https://vnexpress.net/cach-che-bien-vai-mon-au-don-gian-4311539.html
Lẩu hải sản Pháp
Mỳ bò băm
Cơm chiên nho
Hàu sốt bơ chanh
[INFO] Fetched 4 recipes from https://vnexpress.net/nhung-mon-ngon-cho-mua-heo-may-4311484.html
Canh rong biển sườn non
Bò kho ớt
Bí đỏ xào nghêu
Ếch tay cầm
[INFO] Fetched 3 recipes from https://vnexpress.net/3-mon-ngon-de-lam-4311455.html
Khoai tây xào thịt bò
Cá hú kho tiêu xanh
Mít non xào tỏi
[INFO] Fetched 2 recipes from https://vnexpress.net/bien-tau-voi-mon-canh-4311501.html
Canh bắp
Canh mướp trứng
[INFO] Fetched 4 recipes from https://vnexpress.net/cac-mon-an-dam-huong-que-4311479.html
Nem ốc
Xôi rô đồng
Gỏi sứa
Canh ghẹ rau muống
[INFO] Fetched 2 recipes from https://vnexpress.net/hai-mon-ngon-chua-benh-4311475.html
Chè củ cải
Rau cải xào nấm
[INFO] Fetched 5 recipes from https://vnexpress.net/bien-tau-mon-gan-4311449.html
Gan ngỗng xốt dâu tây
Gan ngỗng xốt giấm đen
Gan vịt rán vàng xốt chanh
Gan gà rán muối ớt
Pa-tê ga

In [32]:
all_foods_details

[{'link': 'https://vnexpress.net/cach-che-bien-vai-mon-au-don-gian-4311539.html',
  'type_of_food': 'Món ngon hàng ngày',
  'title': 'Lẩu hải sản Pháp',
  'description': 'Các món ăn Âu dưới đây mang phong cách pha trộn Âu - Á, vừa không làm mất đi hương vị gốc của món ăn vừa không ngán.',
  'author_name': None,
  'cook_time': None,
  'num_of_people': None,
  'calories': None,
  'num_of_ingredients': 9,
  'ingredients': ['Tôm 200 g',
   'nghêu 200 g',
   'mực 200 g',
   'cà 200 g',
   'nghệ tây',
   'cà chua 1 trái',
   'rượu vang 100 ml',
   'nước đường 1 lít',
   'rau mùi 20 g'],
  'step': ['Bước 1: Xào cà chua cho thơm, đổ rượu vang vào đun sắc cho hết cồn, cho tiếp nước dùng vào',
   'Bước 2: Bỏ nghệ tây, rau mùi, nêm gia vị vừa ăn',
   'Bước 3: Khi dùng, đun nước sôi, cho hải sản vào',
   'Bước 4: Dùng với các loại rau ưa thích',
   'Bước 5: Có thể ăn với bún hay mì tùy thích'],
  'note': [],
  'post_date': 'Thứ hai, 22/11/2021, 10:22 (GMT+7)'},
 {'link': 'https://vnexpress.net/cac

In [29]:
len(all_foods_details)

152

In [35]:
all_foods_detail_df = pd.DataFrame(all_foods_details)
#all_foods_detail_df.to_csv('vnexpress_foods_detail_filled.csv', index=False)

In [36]:
all_foods_detail_df

,link,type_of_food,title,description,author_name,cook_time,num_of_people,calories,num_of_ingredients,ingredients,step,note,post_date
0,https://vnexpress.net/cach-che-bien-vai-mon-au...,Món ngon hàng ngày,Lẩu hải sản Pháp,Các món ăn Âu dưới đây mang phong cách pha trộ...,None,None,None,None,9.0,"[Tôm 200 g, nghêu 200 g, mực 200 g, cà 200 g, ...","[Bước 1: Xào cà chua cho thơm, đổ rượu vang và...",[],"Thứ hai, 22/11/2021, 10:22 (GMT+7)"
1,https://vnexpress.net/cach-che-bien-vai-mon-au...,Món ngon hàng ngày,Mỳ bò băm,Các món ăn Âu dưới đây mang phong cách pha trộ...,None,None,None,None,8.0,"[Mỳ Ý 100 g, cà chua 200 g, thịt bò 200 g, hàn...",[Bước 1: Luộc mì Ý trong nước sôi khoảng 8 phú...,[],"Thứ hai, 22/11/2021, 10:22 (GMT+7)"
2,https://vnexpress.net/cach-che-bien-vai-mon-au...,Món ngon hàng ngày,Cơm chiên nho,Các món ăn Âu dưới đây mang phong cách pha trộ...,None,None,None,None,7.0,"[Cơm trắng 200 g, nho 100 g, cà rốt 30 g, bắp ...","[Bước 1: Đặt chảo lên bếp, chờ chảo nóng cho b...",[],"Thứ hai, 22/11/2021, 10:22 (GMT+7)"
3,https://vnexpress.net/cach-che-bien-vai-mon-au...,Món ngon hàng ngày,Hàu sốt bơ chanh,Các món ăn Âu dưới đây mang phong cách pha trộ...,None,None,None,None,6.0,"[Hàu 1 kg, bơ 50 g, chanh 10 g, đường 10 g, ng...","[Bước 1: Hàu rửa sạch, tách vỏ ra làm đôi. Hàn...",[],"Thứ hai, 22/11/2021, 10:22 (GMT+7)"
4,https://vnexpress.net/nhung-mon-ngon-cho-mua-h...,Món ngon hàng ngày,Canh rong biển sườn non,"Canh rong biển sườn non, bò kho ớt, bí đỏ xào ...",None,None,None,None,8.0,"[200 g sườn non, 50 rong biển (loại nấu canh),...","[Bước 1: Sườn non chà muối cho sạch, rửa rồi đ...",[],"Thứ năm, 18/11/2021, 15:44 (GMT+7)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,https://vnexpress.net/sinh-to-tu-rau-qua-43112...,"Món tráng miệng, giải khát",Yoghurt và rau cần,"Ngon, có lợi cho sức khỏe là công dụng c...",None,None,None,None,5.0,"[Nguyên liệu: 300 g rau cần, 1 thìa súp yo...","[Bước 1: Rau cần bỏ lá sâu, rửa sạch. Cà...",[],"Thứ ba, 26/10/2021, 11:18 (GMT+7)"
148,https://vnexpress.net/sinh-to-tu-rau-qua-43112...,"Món tráng miệng, giải khát",Cà rốt và cần tây,"Ngon, có lợi cho sức khỏe là công dụng c...",None,None,None,None,4.0,"[Nguyên liệu: 1 củ cà rốt, 50 g cần tây, ...","[Bước 1: Cà rốt bào vỏ, rửa sạch, thái ...",[],"Thứ ba, 26/10/2021, 11:18 (GMT+7)"
149,https://vnexpress.net/sinh-to-tu-rau-qua-43112...,"Món tráng miệng, giải khát",Đu đủ và bắp cải tím,"Ngon, có lợi cho sức khỏe là công dụng c...",None,None,None,None,6.0,"[Nguyên liệu: 1/4 quả đu đủ, 20g bắp cải ...","[Bước 1: Đu đủ gọt bỏ vỏ, hạt, thái miế...",[],"Thứ ba, 26/10/2021, 11:18 (GMT+7)"
150,https://vnexpress.net/mon-trang-mieng-4311313....,"Món tráng miệng, giải khát",Dưa hấu trộn mật ong,Dưa hấu trộn mật ong và đào ngâm rượu vang là ...,None,None,None,None,3.0,"[1 kg dưa hấu, 5 muỗng cà phê mật ong, muối ti...","[Bước 1: Dưa hấu gọt vỏ, bỏ hạt, xắt hạt lựu (...",[],"Thứ ba, 26/10/2021, 11:01 (GMT+7)"


### Merge data

In [37]:
raw_data_df = pd.read_csv('vnexpress_foods_detail.csv') 
filtered_data_df = all_foods_detail_df


# filter out rows in A with empty 'step'
mask = raw_data_df.apply(
    lambda row: row['step'] in ([], "[]"),
    axis=1
)
raw_data_df_filtered = raw_data_df[~mask].reset_index(drop=True)

# check duplicate urls between A and B
# get urls in A
existing_urls = set(raw_data_df_filtered['link'])

# Keep only rows in B whose 'url' is not in A
filtered_data_df_unique = filtered_data_df[~filtered_data_df['link'].isin(existing_urls)].reset_index(drop=True)

# concatenate
final_table = pd.concat([raw_data_df_filtered, filtered_data_df_unique], ignore_index=True)

final_table.to_csv('vnexpress_foods_detail_merged.csv', index=False)

print(f"""
- Old table A: {len(raw_data_df)}
- Removed empty step rows: {mask.sum()}
- Table B (filled): {len(filtered_data_df)}
- Duplicates removed from B: {len(filtered_data_df) - len(filtered_data_df_unique)}
- Final merged table: {len(final_table)}
""")




- Old table A: 810
- Removed empty step rows: 69
- Table B (filled): 152
- Duplicates removed from B: 0
- Final merged table: 893

